In [1]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import psycopg2
import calendar
import random
import seaborn as sns
import numpy as np
import itertools
import warnings
warnings.filterwarnings("ignore")

In [2]:
conn = psycopg2.connect(
  port='5432',
  host= "shopify-merchant-dump.ccm9mnr5avgs.ap-south-1.rds.amazonaws.com",
  user= "softtouchlenses_4e0b7c925f8444429457581f35678aea",
  password="softtouchlenses_4e0b7c925f8444429457581f35678aea_password",
  database= "softtouchlenses_4e0b7c925f8444429457581f35678aea_db"
)

In [3]:
query1 = """select o.id as order_id,
            o.created_at::date as order_date,
            o.customer_id as user_id,
            oli.product_id as product_id,
            oli.name as product_name,
            o.total_price as order_amount,
            CASE
        WHEN cancelled_at IS NOT NULL THEN 'Cancelled'
         ELSE 'Delivered'
     END as order_status,
            oli.quantity as quantity,
            oli.price as sp
            from orders o
            join order_line_item oli on oli.order_id = o.id"""

In [4]:
data = pd.read_sql(query1, conn)

In [5]:
data["order_date"] = pd.to_datetime(data["order_date"])
data[data["product_id"] == np.nan]
data = data.dropna()
data['WeekCount']=data['order_date'].dt.strftime('%Y-w%U')
data=data.sort_values('order_date')
WeekList=data['WeekCount'].unique()

In [6]:
WeekList = WeekList.tolist()
output_list = []

for i in range(len(WeekList) - 1):
    output_list.append(WeekList[i])
    current_week = int(WeekList[i].split('-w')[1])
    next_week = int(WeekList[i + 1].split('-w')[1])
    missing_weeks = next_week - current_week - 1
    for j in range(1, missing_weeks + 1):
        missing_week = current_week + j
        output_list.append(f'{WeekList[i][:5]}w{missing_week:02d}')

output_list.append(WeekList[-1])

In [7]:
data = data[data['order_status'] != 'Cancelled']

In [8]:
orderuserlevel=data.groupby(['order_id','user_id']).agg(
          order_date=pd.NamedAgg(column='order_date', aggfunc='min'),

    ).reset_index()
orderuserlevel=orderuserlevel.sort_values('order_date')
orderuserlevel['TrxnRank'] = orderuserlevel.groupby('user_id')['order_date'].rank(method='first')
orderuserlevel


data=pd.merge(data,
         orderuserlevel[['order_id','TrxnRank']],
         on="order_id",
         how ='left')
data['New']=0
data['Repeat']=0
data.loc[data['TrxnRank']==1,'New']=1
data.loc[data['TrxnRank']>1,'Repeat']=1

#AOV Contribution
data['ordervalue']=data['quantity']*data['sp']
data['AOVContribution']=data['ordervalue']/data["order_id"].nunique()
data['DeltaDaysinOrders']=(data.groupby('user_id').order_date.shift() - data.order_date).dt.days.abs()
data['DeltaOrderValues']=(data.groupby('user_id').ordervalue.shift() - data.ordervalue)

In [9]:
Weekly_Stats = {}
productsegstats_weekly = {}
user_cohort_labels = ['New', 'Churned (Single Use Only)', 'Recent & Loyals', 'Champions', 'Churned (Repeat Use)']
user_cohort_var_list = ['Users']#, 'Orders', 'Recency', 'AOVContribution', 'OrderValue','MeanDaysDelta', 'MeanOVDelta', 'Maturity', 'Frequency', 'AOV'
product_cohort_labels=['Hero', 'High Value & Slow Moving', 'Low Value & Slow Moving', 'New Business Drivers', 'Repeat Business Drivers']
product_cohort_var_list=['ProductCount'] #'Orders','Users','AverageSellingPrice','QuantityperCart','CartPenetration','NewUserRatio','NewOrders','ASPXQpC','AOVContribution','ProductCountMix','OrdersMix','UsersMix','AOVContribution(%)','OrderPerSKU'
final_stats = pd.DataFrame()

for x in output_list:
  x3 = data[data['WeekCount'] <= x]
  try:
    userweeklydata = x3.groupby("user_id").agg(
          Last_Date=pd.NamedAgg(column='order_date', aggfunc= lambda x: x.nlargest(2).min()),
          First_Date=pd.NamedAgg(column='order_date', aggfunc='min'),
          OrdersCount=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          Frequency=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
          #Mean_Days_Delta=pd.NamedAgg(column='DeltaDaysinOrders', aggfunc='mean'),
          #Mean_OV_Delta=pd.NamedAgg(column='DeltaOrderValues', aggfunc='mean')
          ).reset_index()

    globalmaxdateweek=x3['order_date'].max()
    userweeklydata['Recency']=globalmaxdateweek-userweeklydata['Last_Date']
    userweeklydata['Recency']=userweeklydata['Recency'].dt.days.abs()
    userweeklydata['MaturityAge']=np.nan
    userweeklydata.loc[userweeklydata['Frequency']>1,"MaturityAge"]=(userweeklydata['Last_Date']-userweeklydata['First_Date']).dt.days.abs()
    #userweeklydata['Mean_Days_Delta']=userweeklydata['Mean_Days_Delta'].fillna(0)
    #userweeklydata['Mean_OV_Delta']=userweeklydata['Mean_OV_Delta'].fillna(0)
    userweeklydata['Customer_AOV']=userweeklydata['Order_Value']/userweeklydata['OrdersCount']

    r_labels, f_labels,AOV_labels, m_labels,DelAOV_labels = range(4, 0, -1), range(1,5), range(1,5),range(1,5),range(1,5)
    #userweeklydata['f_score'] = userweeklydata['OrdersCount'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = f_labels)).astype('int64')
    userweeklydata['r_score'] = userweeklydata['Recency'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = r_labels)).astype('int64')
   # userweeklydata['AOV_score'] = userweeklydata['Customer_AOV'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = AOV_labels)).astype('int64')
    userweeklydata['m_score']=0
    userweeklydata.loc[userweeklydata['MaturityAge'].notnull(),'m_score']= userweeklydata.loc[userweeklydata['MaturityAge'].notnull()]['MaturityAge'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = m_labels)).astype('int64')
    #userweeklydata['DelAOV'] = userweeklydata['Mean_OV_Delta'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = DelAOV_labels)).astype('int64')
    #userweeklydata['SumScore'] = userweeklydata['r_score'] + userweeklydata['m_score'] + userweeklydata['AOV_score']+ userweeklydata['DelAOV'] +userweeklydata['f_score']
    def CohortLables_3(df):
      if ((df['Frequency']==1) & (df['r_score']>2)):
          return "New"
      elif ((df['Frequency'] == 1) & (df['r_score'] <= 2)):
          return 'Churned (Single Use Only)'
      elif ((df['Frequency'] > 1) & (df['m_score'] < 3) & (df['r_score'] >2)):
          return 'Recent & Loyals'
      elif ((df['Frequency'] > 1) & (df['m_score'] >=3) & (df['r_score'] > 2)):
          return 'Champions'
      else:
          return 'Churned (Repeat Use)'
    userweeklydata['User_Cohort_Label'] = userweeklydata.apply(CohortLables_3, axis=1)
    x3=pd.merge(x3,
         userweeklydata[['user_id','User_Cohort_Label']],
         on="user_id",
         how ='left')

    totalcarts=x3['order_id'].nunique()
    productlevel=x3.groupby(['product_id','product_name']).agg(
              Carts=pd.NamedAgg(column='order_id', aggfunc='nunique'),
              quantity=pd.NamedAgg(column='quantity', aggfunc='sum'),
              ASP=pd.NamedAgg(column='sp', aggfunc='mean'),
              NewOrders=pd.NamedAgg(column='New', aggfunc='sum'),
              RepeatOrders=pd.NamedAgg(column='Repeat', aggfunc='sum'),
              Users=pd.NamedAgg(column='user_id', aggfunc='nunique'),
              AOVContribution=pd.NamedAgg(column='AOVContribution', aggfunc='sum'),
        ).reset_index()
    productlevel['CartPenetration']=productlevel['Carts']/totalcarts
    productlevel['NewUserRatio']=productlevel['NewOrders']/productlevel['Users']
    productlevel['QpC']=productlevel['quantity']/productlevel['Carts']
    productlevel['ASPXQpC']=productlevel['ASP']*productlevel['QpC']

    CP_labels, NewRatio_labels,ASP_labels, QpC_labels =  range(1,5), range(1,5),range(1,5),range(1,5)

    productlevel['CP_score'] = productlevel['CartPenetration'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = CP_labels)).astype('int64')
    productlevel['NewRatio_score'] = productlevel['NewUserRatio'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = NewRatio_labels)).astype('int64')
    productlevel['ASP_score'] = productlevel['ASP'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = ASP_labels)).astype('int64')
    productlevel['QpC_score'] = productlevel['QpC'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = QpC_labels)).astype('int64')
    productlevel['SumScore'] = productlevel['CP_score'] + productlevel['NewRatio_score'] + productlevel['ASP_score'] + productlevel['QpC_score']

    def ProductCohortLables2(df):
      if ((df['CP_score']>2) and (df['NewRatio_score'] >3)):
          return 'New Business Drivers'
      elif ((df['CP_score'] >2) and (df['NewRatio_score'] <2)):
          return 'Repeat Business Drivers'
      elif (df['CP_score'] >2):
          return 'Hero'
      elif (df['ASP_score'] >3):
          return 'High Value & Slow Moving'
      elif (df['ASP_score'] <=3):
          return 'Low Value & Slow Moving'
      else:
          return 'Low Value & Slow Moving'

    productlevel['Product Cohort Lables'] = productlevel.apply(ProductCohortLables2, axis=1)
    x3=pd.merge(x3,
         productlevel[['product_id','Product Cohort Lables']],
         on="product_id",
         how ='left')


    User_Product_Level=x3.groupby(['User_Cohort_Label','Product Cohort Lables']).agg(
        Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
            Users=pd.NamedAgg(column='user_id', aggfunc='nunique'),
        Order_Count=pd.NamedAgg(column='order_id', aggfunc='nunique'),

    ).reset_index()

    Weekly_Stats[x] = User_Product_Level
    x4 = data[data['WeekCount']==x]
    x5 = pd.merge(x4,userweeklydata[['user_id','User_Cohort_Label']], on='user_id', how='left')
    x5 = pd.merge(x5,productlevel[['product_id','Product Cohort Lables']], on='product_id', how='left')
    final_stats = final_stats.append(x5, ignore_index=True)
  except:
    pass


In [10]:
final_stats

,order_id,order_date,user_id,product_id,product_name,order_amount,order_status,quantity,sp,WeekCount,TrxnRank,New,Repeat,ordervalue,AOVContribution,DeltaDaysinOrders,DeltaOrderValues,User_Cohort_Label,Product Cohort Lables
0,3969073283246,2021-08-19,5425893015726,7.251689e+12,Affaires Magnetic Reading Spectacle Glasses Fo...,599.0,Delivered,1,599.0,2021-w33,1.0,1,0,599.0,0.257634,NaN,NaN,Churned (Single Use Only),Low Value & Slow Moving
1,3969315209390,2021-08-19,5426204672174,7.253567e+12,SOFLENS 59 - BAUSCH & LOMB (Monthly) (6 Lenses...,899.0,Delivered,1,899.0,2021-w33,1.0,1,0,899.0,0.386667,NaN,NaN,Churned (Single Use Only),New Business Drivers
2,3970425356462,2021-08-20,5427174932654,7.249161e+12,Power Swimming Goggles Aryca Rx Prescription O...,1399.0,Delivered,1,1399.0,2021-w33,1.0,1,0,1399.0,0.601720,NaN,NaN,New,High Value & Slow Moving
3,3972209148078,2021-08-20,5430117630126,7.253530e+12,Optima 38 DAILY WEAR LENS (1 Lens per Box) Bau...,1500.0,Delivered,2,450.0,2021-w33,1.0,1,0,900.0,0.387097,NaN,NaN,New,Hero
4,3972209148078,2021-08-20,5430117630126,7.253349e+12,Affaires Clear UV Contact Lenses Yearly Dispos...,1500.0,Delivered,1,600.0,2021-w33,1.0,1,0,600.0,0.258065,0.0,300.0,New,Low Value & Slow Moving
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28418,5231961374894,2023-12-15,6773256618158,7.253438e+12,Affaires Color Yearly Contact Lenses Two Tone ...,900.0,Delivered,1,900.0,2023-w50,1.0,1,0,900.0,0.387097,NaN,NaN,New,Low Value & Slow Moving
28419,5231961374894,2023-12-15,6773256618158,7.253438e+12,Affaires Color Yearly Contact Lenses Two Tone ...,900.0,Delivered,1,900.0,2023-w50,1.0,1,0,900.0,0.387097,NaN,NaN,New,Low Value & Slow Moving
28420,5231961374894,2023-12-15,6773256618158,7.253438e+12,Affaires Color Yearly Contact Lenses Two Tone ...,900.0,Delivered,1,900.0,2023-w50,1.0,1,0,900.0,0.387097,NaN,NaN,New,Low Value & Slow Moving
28421,5231961374894,2023-12-15,6773256618158,7.253438e+12,Affaires Color Yearly Contact Lenses Two Tone ...,900.0,Delivered,1,900.0,2023-w50,1.0,1,0,900.0,0.387097,NaN,NaN,New,Hero


In [11]:
#AOV Contribution
final_stats['ordervalue']=final_stats['ordervalue']
final_stats['AOVContribution']=final_stats['ordervalue']/final_stats["order_id"].nunique()
final_stats['DeltaDaysinOrders']=(final_stats.groupby('user_id').order_date.shift() - final_stats.order_date).dt.days.abs()
final_stats['DeltaOrderValues']=(final_stats.groupby('user_id').ordervalue.shift() - final_stats.ordervalue)
weekly_summary_stats = final_stats.groupby(['WeekCount','User_Cohort_Label','Product Cohort Lables']).agg(
            totalorders=pd.NamedAgg(column='order_id', aggfunc='nunique'),
            totalusers=pd.NamedAgg(column='user_id', aggfunc='nunique'),
            totalvalue=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
            quantity=pd.NamedAgg(column='quantity', aggfunc='sum'),
            SKUCounts=pd.NamedAgg(column='product_id', aggfunc='nunique')

      ).reset_index()
newdf=final_stats[final_stats['New']==1].groupby(['WeekCount','User_Cohort_Label','Product Cohort Lables']).agg(
    newusercount=pd.NamedAgg(column='user_id', aggfunc='nunique'),
     ).reset_index()
cancelled_orders = final_stats[final_stats['order_status'] == 'Cancelled'].groupby(['WeekCount','User_Cohort_Label','Product Cohort Lables']).agg(
    CancellationCount=pd.NamedAgg(column='order_id', aggfunc='count')
).fillna(0).reset_index()

delivered_orders = final_stats[final_stats['order_status'] == 'Delivered'].groupby(['WeekCount','User_Cohort_Label','Product Cohort Lables']).agg(
    DeliveryCount=pd.NamedAgg(column='order_id', aggfunc='count')
).fillna(0).reset_index()
weekly_summary_stats = weekly_summary_stats.merge(newdf, on=['WeekCount','User_Cohort_Label','Product Cohort Lables'], how="left")
weekly_summary_stats =weekly_summary_stats.merge(cancelled_orders, on=['WeekCount','User_Cohort_Label','Product Cohort Lables'], how="left")
weekly_summary_stats = weekly_summary_stats.merge(delivered_orders, on=['WeekCount','User_Cohort_Label','Product Cohort Lables'], how="left")
weekly_summary_stats['repeatusercount']=weekly_summary_stats['totalusers']-weekly_summary_stats['newusercount']
weekly_summary_stats['AOV'] = weekly_summary_stats['totalvalue']/ weekly_summary_stats['totalorders']


In [12]:
weekly_summary_stats

,WeekCount,User_Cohort_Label,Product Cohort Lables,totalorders,totalusers,totalvalue,quantity,SKUCounts,newusercount,CancellationCount,DeliveryCount,repeatusercount,AOV
0,2021-w33,Churned (Single Use Only),Low Value & Slow Moving,1,1,599.0,1,1,1.0,NaN,1,0.0,599.000000
1,2021-w33,Churned (Single Use Only),New Business Drivers,1,1,899.0,1,1,1.0,NaN,1,0.0,899.000000
2,2021-w33,New,Hero,1,1,900.0,2,1,1.0,NaN,1,0.0,900.000000
3,2021-w33,New,High Value & Slow Moving,1,1,1399.0,1,1,1.0,NaN,1,0.0,1399.000000
4,2021-w33,New,Low Value & Slow Moving,1,1,600.0,1,1,1.0,NaN,1,0.0,600.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1040,2023-w50,New,Low Value & Slow Moving,12,12,41091.0,46,7,12.0,NaN,46,0.0,3424.250000
1041,2023-w50,New,New Business Drivers,7,7,19178.0,18,4,7.0,NaN,18,0.0,2739.714286
1042,2023-w50,New,Repeat Business Drivers,18,18,263552.0,194,11,18.0,NaN,193,0.0,14641.777778
1043,2023-w50,Recent & Loyals,High Value & Slow Moving,1,1,38320.0,16,1,NaN,NaN,4,NaN,38320.000000


In [13]:
final_stats["Retention"] = 0
final_stats.loc[(final_stats["DeltaDaysinOrders"] > 0) & (final_stats["DeltaDaysinOrders"] <= 90) , "Retention"] = "Retained"
final_stats.loc[(final_stats["DeltaDaysinOrders"] > 90), "Retention"] = "Not-Retained"
retained_df = final_stats[final_stats['Retention'] == 'Retained'].groupby(['WeekCount','User_Cohort_Label','Product Cohort Lables']).size().reset_index(name='Retained_Users')
weekly_summary_stats = pd.merge(weekly_summary_stats, retained_df, on=['WeekCount','User_Cohort_Label','Product Cohort Lables'], how="left")

In [14]:
key_metrices = ['WeekCount','User_Cohort_Label','Product Cohort Lables','totalorders', 'totalvalue', 'repeatusercount','newusercount', 'AOV']
weekly_summary_stats = weekly_summary_stats[key_metrices]

weekly_summary_stats = weekly_summary_stats.rename(columns={
    'totalorders':'Total_Orders',
    'totalvalue': 'Total_Value',
    'repeatusercount': 'No_of_Repeat_Users',
    'newusercount': 'No_of_New_Users(First_Time_Users)',
    'AOV': 'Average_Order_Value'
})
weekly_summary_stats=weekly_summary_stats.fillna(0)

In [15]:
weekly_summary_stats['AOV_Contribution'] = weekly_summary_stats['Total_Value'] / weekly_summary_stats.groupby('WeekCount')['Total_Orders'].transform('sum')

In [16]:
grouped_by_week = weekly_summary_stats.groupby('WeekCount')
dataframes_by_week = {}
for week, group in grouped_by_week:
    dataframes_by_week[week] = group.drop(columns='WeekCount').copy()

In [17]:
weekly_summary_stats

,WeekCount,User_Cohort_Label,Product Cohort Lables,Total_Orders,Total_Value,No_of_Repeat_Users,No_of_New_Users(First_Time_Users),Average_Order_Value,AOV_Contribution
0,2021-w33,Churned (Single Use Only),Low Value & Slow Moving,1,599.0,0.0,1.0,599.000000,119.800000
1,2021-w33,Churned (Single Use Only),New Business Drivers,1,899.0,0.0,1.0,899.000000,179.800000
2,2021-w33,New,Hero,1,900.0,0.0,1.0,900.000000,180.000000
3,2021-w33,New,High Value & Slow Moving,1,1399.0,0.0,1.0,1399.000000,279.800000
4,2021-w33,New,Low Value & Slow Moving,1,600.0,0.0,1.0,600.000000,120.000000
...,...,...,...,...,...,...,...,...,...
1040,2023-w50,New,Low Value & Slow Moving,12,41091.0,0.0,12.0,3424.250000,578.746479
1041,2023-w50,New,New Business Drivers,7,19178.0,0.0,7.0,2739.714286,270.112676
1042,2023-w50,New,Repeat Business Drivers,18,263552.0,0.0,18.0,14641.777778,3712.000000
1043,2023-w50,Recent & Loyals,High Value & Slow Moving,1,38320.0,0.0,0.0,38320.000000,539.718310


In [18]:
dataframes_by_week

{'2021-w33':            User_Cohort_Label     Product Cohort Lables  Total_Orders  \
 0  Churned (Single Use Only)   Low Value & Slow Moving             1   
 1  Churned (Single Use Only)      New Business Drivers             1   
 2                        New                      Hero             1   
 3                        New  High Value & Slow Moving             1   
 4                        New   Low Value & Slow Moving             1   
 
    Total_Value  No_of_Repeat_Users  No_of_New_Users(First_Time_Users)  \
 0        599.0                 0.0                                1.0   
 1        899.0                 0.0                                1.0   
 2        900.0                 0.0                                1.0   
 3       1399.0                 0.0                                1.0   
 4        600.0                 0.0                                1.0   
 
    Average_Order_Value  AOV_Contribution  
 0                599.0             119.8  
 1            

In [19]:
weekly_summary_stats["key"] = weekly_summary_stats["User_Cohort_Label"] + "-" + weekly_summary_stats["Product Cohort Lables"]
weekly_summary_stats.drop(['User_Cohort_Label', 'Product Cohort Lables'], axis=1, inplace=True)
weekly_summary_stats.set_index("WeekCount", inplace=True)

In [20]:
grouped_dict = {week: group for week, group in weekly_summary_stats.groupby(level='WeekCount')}
for week, df in grouped_dict.items():
    df = df.reset_index(drop=True)
    df.set_index('key', inplace=True)
    grouped_dict[week] = df

In [21]:
def week_metric_user_product_split(week, parameter):
  week_df = dataframes_by_week[week]
  pivot_df = week_df.pivot_table(index='User_Cohort_Label', columns='Product Cohort Lables', values=parameter, aggfunc='sum')
  return pivot_df
df_output_Week = week_metric_user_product_split("2023-w44", "Total_Value")

In [22]:
df_output_Week

Product Cohort Lables,Hero,High Value & Slow Moving,Low Value & Slow Moving,New Business Drivers,Repeat Business Drivers
User_Cohort_Label,,,,,
Champions,74668.0,156240.0,9100.0,210621.0,273989.0
Churned (Repeat Use),39000.0,50832.0,23600.0,77148.0,96692.0
New,260243.0,129423.0,39183.0,33727.0,168903.0
Recent & Loyals,24784.0,9858.0,32062.0,998.0,3797.0


In [23]:
all_combinations = list(itertools.product(user_cohort_labels, product_cohort_labels))
formatted_combinations = [f'{user}-{product}' for user, product in all_combinations]

In [24]:
averages_dict = {}
weeks = list(grouped_dict.keys())

for i in range(8, len(weeks)):
    week = weeks[i]
    df = grouped_dict[week]
    avg_df = df.rolling(window=8, min_periods=1).mean().shift(1)
    avg_df = avg_df.iloc[1:]
    averages_dict[week] = avg_df

In [25]:
averages_dict

{'2021-w42':                                    Total_Orders  Total_Value  \
 key                                                            
 Champions-Low Value & Slow Moving           1.0  1399.000000   
 Champions-Repeat Business Drivers           1.0  2798.000000   
 New-Low Value & Slow Moving                 1.0  2331.666667   
 New-New Business Drivers                    1.0  2225.750000   
 
                                    No_of_Repeat_Users  \
 key                                                     
 Champions-Low Value & Slow Moving                 0.0   
 Champions-Repeat Business Drivers                 0.0   
 New-Low Value & Slow Moving                       0.0   
 New-New Business Drivers                          0.0   
 
                                    No_of_New_Users(First_Time_Users)  \
 key                                                                    
 Champions-Low Value & Slow Moving                               0.00   
 Champions-Repeat Business 

In [26]:
for key, df in averages_dict.items():
    missing_indices = [index for index in formatted_combinations if index not in df.index]
    if missing_indices:
        for missing_index in missing_indices:
            df.loc[missing_index] = 0

In [27]:
grouped_dict

{'2021-w33':                                                    Total_Orders  Total_Value  \
 key                                                                            
 Churned (Single Use Only)-Low Value & Slow Moving             1        599.0   
 Churned (Single Use Only)-New Business Drivers                1        899.0   
 New-Hero                                                      1        900.0   
 New-High Value & Slow Moving                                  1       1399.0   
 New-Low Value & Slow Moving                                   1        600.0   
 
                                                    No_of_Repeat_Users  \
 key                                                                     
 Churned (Single Use Only)-Low Value & Slow Moving                 0.0   
 Churned (Single Use Only)-New Business Drivers                    0.0   
 New-Hero                                                          0.0   
 New-High Value & Slow Moving                    

In [28]:
result_list = []

for key, dataframe in grouped_dict.items():
    sum_result = pd.DataFrame(dataframe.sum(), columns=['sum_result']).T
    sum_result['key'] = key
    sum_result['original_index'] = dataframe.index[0]
    sum_result.set_index('original_index', inplace=True)
    result_list.append(sum_result)

result_df_1= pd.concat(result_list, ignore_index=True)

In [29]:
result_df_melted_1 = pd.melt(result_df_1, id_vars=['key'], var_name='Metric', value_name='Value')
result_df_melted_1.rename(columns={'key': 'WeekCount'}, inplace=True)

In [30]:
result_df_melted_1["Metric"].unique()

array(['Total_Orders', 'Total_Value', 'No_of_Repeat_Users',
       'No_of_New_Users(First_Time_Users)', 'Average_Order_Value',
       'AOV_Contribution'], dtype=object)

In [31]:
result_list = []

for key, dataframe in averages_dict.items():
    sum_result = pd.DataFrame(dataframe.sum(), columns=['sum_result']).T  # Convert sum_result to a DataFrame and transpose
    sum_result['key'] = key
    sum_result['original_index'] = dataframe.index[0]  # Use the first index value
    sum_result.set_index('original_index', inplace=True)
    result_list.append(sum_result)
result_df_2= pd.concat(result_list, ignore_index=True)

In [32]:
result_df_melted_2 = pd.melt(result_df_2, id_vars=['key'], var_name='Metric', value_name='Value')
result_df_melted_2.rename(columns={'key': 'WeekCount'}, inplace=True)

In [33]:
result_df_melted_2

,WeekCount,Metric,Value
0,2021-w42,Total_Orders,4.000000
1,2021-w43,Total_Orders,6.500000
2,2021-w44,Total_Orders,6.666667
3,2021-w45,Total_Orders,12.550000
4,2021-w46,Total_Orders,4.250000
...,...,...,...
673,2023-w46,AOV_Contribution,12381.924320
674,2023-w47,AOV_Contribution,7687.566993
675,2023-w48,AOV_Contribution,19481.049315
676,2023-w49,AOV_Contribution,7251.701583


In [34]:
def create_metric_df(data_dict, metric_column):
    curr_week_df = pd.DataFrame()

    for week, dataframe in data_dict.items():
        current_index_list = dataframe.index.tolist()
        current_metric_values = dataframe[metric_column].tolist()

        temp_df = pd.DataFrame({
            'Metric': current_index_list,
            'Value': current_metric_values,
            'WeekCount': [week] * len(current_index_list)
        })
        curr_week_df = curr_week_df.append(temp_df, ignore_index=True)

    curr_week_df = curr_week_df.rename(columns={
        'Metric': 'Metric',
        'Value': 'Value',
        'WeekCount': 'WeekCount'
    })

    return curr_week_df
curr_week_df = create_metric_df(grouped_dict, 'Total_Orders')

In [35]:
curr_week_df

,Metric,Value,WeekCount
0,Churned (Single Use Only)-Low Value & Slow Moving,1,2021-w33
1,Churned (Single Use Only)-New Business Drivers,1,2021-w33
2,New-Hero,1,2021-w33
3,New-High Value & Slow Moving,1,2021-w33
4,New-Low Value & Slow Moving,1,2021-w33
...,...,...,...
1040,New-Low Value & Slow Moving,12,2023-w50
1041,New-New Business Drivers,7,2023-w50
1042,New-Repeat Business Drivers,18,2023-w50
1043,Recent & Loyals-High Value & Slow Moving,1,2023-w50


In [36]:
def create_average_df(data_dict, metric_column):

    avg_week_df = pd.DataFrame()

    for week, dataframe in data_dict.items():
        current_index_list = dataframe.index.tolist()
        current_metric_values = dataframe[metric_column].tolist()

        temp_df = pd.DataFrame({
            'Metric': current_index_list,
            'Value': current_metric_values,
            'WeekCount': [week] * len(current_index_list)
        })
        avg_week_df = avg_week_df.append(temp_df, ignore_index=True)

    avg_week_df = avg_week_df.rename(columns={
        'Metric': 'Metric',
        'Value': 'Value',
        'WeekCount': 'WeekCount'
    })
    return avg_week_df

avg_week_df = create_average_df(averages_dict, 'Total_Value')


In [37]:
avg_week_df

,Metric,Value,WeekCount
0,Champions-Low Value & Slow Moving,1399.000000,2021-w42
1,Champions-Repeat Business Drivers,2798.000000,2021-w42
2,New-Low Value & Slow Moving,2331.666667,2021-w42
3,New-New Business Drivers,2225.750000,2021-w42
4,New-Hero,0.000000,2021-w42
...,...,...,...
2820,Churned (Repeat Use)-Hero,0.000000,2023-w50
2821,Churned (Repeat Use)-High Value & Slow Moving,0.000000,2023-w50
2822,Churned (Repeat Use)-Low Value & Slow Moving,0.000000,2023-w50
2823,Churned (Repeat Use)-New Business Drivers,0.000000,2023-w50


In [38]:
current_week_dataframe = pd.concat([result_df_melted_1, curr_week_df], axis=0)
current_week_dataframe = current_week_dataframe.sort_values(by='WeekCount', ascending=True)
current_week_dataframe.rename(columns={'Metric': 'Variables', 'Value' : 'Current Week Value'}, inplace=True)
keys_list = list(averages_dict.keys())
current_week_dataframe = current_week_dataframe[current_week_dataframe['WeekCount'].isin(keys_list)]

In [39]:
current_week_dataframe[current_week_dataframe["WeekCount"] == "2023-w50"]

,WeekCount,Variables,Current Week Value
1038,2023-w50,New-Hero,5.00000
1042,2023-w50,New-Repeat Business Drivers,18.00000
1041,2023-w50,New-New Business Drivers,7.00000
1036,2023-w50,Champions-New Business Drivers,4.00000
1040,2023-w50,New-Low Value & Slow Moving,12.00000
1039,2023-w50,New-High Value & Slow Moving,6.00000
1037,2023-w50,Champions-Repeat Business Drivers,6.00000
1035,2023-w50,Champions-Low Value & Slow Moving,5.00000
1043,2023-w50,Recent & Loyals-High Value & Slow Moving,1.00000
1033,2023-w50,Champions-Hero,4.00000


In [40]:
last_8_week_dataframe = pd.concat([result_df_melted_2, avg_week_df], axis=0)
last_8_week_dataframe = last_8_week_dataframe.sort_values(by='WeekCount', ascending=True)
last_8_week_dataframe.rename(columns={'Metric': 'Variables', 'Value' : 'Last 8 Week Value'}, inplace=True)

In [41]:
last_8_week_dataframe

,WeekCount,Variables,Last 8 Week Value
0,2021-w42,Total_Orders,4.000000
22,2021-w42,Churned (Repeat Use)-Low Value & Slow Moving,0.000000
21,2021-w42,Churned (Repeat Use)-High Value & Slow Moving,0.000000
20,2021-w42,Churned (Repeat Use)-Hero,0.000000
19,2021-w42,Champions-New Business Drivers,0.000000
...,...,...,...
112,2023-w50,Total_Orders,52.303571
2823,2023-w50,Churned (Repeat Use)-New Business Drivers,0.000000
677,2023-w50,AOV_Contribution,6311.684440
2808,2023-w50,New-Repeat Business Drivers,30783.125000


In [42]:
merged_dataframe = pd.merge(current_week_dataframe, last_8_week_dataframe, on=['WeekCount', 'Variables'], how='left')
merged_dataframe = merged_dataframe.fillna(0)
primary_explanatory_vars = ['Total_Orders', 'No_of_Repeat_Users', 'AOV_Contribution', 'No_of_New_Users(First_Time_Users)', '90_Day_Retention']
target_vars = ["Total_Value"]
merged_dataframe.loc[merged_dataframe['Variables'].isin(target_vars), 'Tag'] = 'Target_Variable'
merged_dataframe.loc[merged_dataframe['Variables'].isin(primary_explanatory_vars), 'Tag'] = 'Primary_Explanatory'
merged_dataframe.loc[merged_dataframe['Variables'].isin(formatted_combinations), 'Tag'] = 'Secondary_Explanatory'
merged_dataframe['Difference'] = merged_dataframe['Current Week Value'] - merged_dataframe['Last 8 Week Value']

In [43]:
merged_dataframe[merged_dataframe["WeekCount"] == "2021-w42"]

,WeekCount,Variables,Current Week Value,Last 8 Week Value,Tag,Difference
0,2021-w42,Champions-High Value & Slow Moving,1.000000,0.000000,Secondary_Explanatory,1.000000
1,2021-w42,Champions-Low Value & Slow Moving,1.000000,1399.000000,Secondary_Explanatory,-1398.000000
2,2021-w42,Champions-Repeat Business Drivers,1.000000,2798.000000,Secondary_Explanatory,-2797.000000
3,2021-w42,New-Low Value & Slow Moving,1.000000,2331.666667,Secondary_Explanatory,-2330.666667
4,2021-w42,Total_Value,12101.000000,8754.416667,Target_Variable,3346.583333
5,2021-w42,New-New Business Drivers,3.000000,2225.750000,Secondary_Explanatory,-2222.750000
6,2021-w42,No_of_Repeat_Users,0.000000,0.000000,Primary_Explanatory,0.000000
7,2021-w42,No_of_New_Users(First_Time_Users),4.000000,0.250000,Primary_Explanatory,3.750000
8,2021-w42,Average_Order_Value,9969.000000,8754.416667,NaN,1214.583333
9,2021-w42,AOV_Contribution,1728.714286,1250.630952,Primary_Explanatory,478.083333


In [44]:
new_df_1 = merged_dataframe[merged_dataframe["Tag"] == "Secondary_Explanatory"]
new_df_1

,WeekCount,Variables,Current Week Value,Last 8 Week Value,Tag,Difference
0,2021-w42,Champions-High Value & Slow Moving,1.0,0.000000,Secondary_Explanatory,1.000000
1,2021-w42,Champions-Low Value & Slow Moving,1.0,1399.000000,Secondary_Explanatory,-1398.000000
2,2021-w42,Champions-Repeat Business Drivers,1.0,2798.000000,Secondary_Explanatory,-2797.000000
3,2021-w42,New-Low Value & Slow Moving,1.0,2331.666667,Secondary_Explanatory,-2330.666667
5,2021-w42,New-New Business Drivers,3.0,2225.750000,Secondary_Explanatory,-2222.750000
...,...,...,...,...,...,...
1669,2023-w50,Champions-Low Value & Slow Moving,5.0,34357.000000,Secondary_Explanatory,-34352.000000
1670,2023-w50,Recent & Loyals-High Value & Slow Moving,1.0,62565.375000,Secondary_Explanatory,-62564.375000
1671,2023-w50,Champions-Hero,4.0,0.000000,Secondary_Explanatory,4.000000
1678,2023-w50,Champions-High Value & Slow Moving,1.0,59420.000000,Secondary_Explanatory,-59419.000000


In [45]:
new_df_1[['User_Cohort_Lables', 'Product_Cohort_Lables']] = new_df_1['Variables'].str.split('-', expand=True)

In [46]:
grouped_by_week_delta = new_df_1.groupby('WeekCount')
dataframes_by_week = {}
for week, group in grouped_by_week_delta:
    dataframes_by_week[week] = group.drop(columns='WeekCount').copy()

In [47]:
def week_metric_user_product_split(week, parameter):
  week_df = dataframes_by_week[week]
  pivot_df = week_df.pivot_table(index='User_Cohort_Lables', columns='Product_Cohort_Lables', values=parameter, aggfunc='sum')
  return pivot_df
df_output_delta = week_metric_user_product_split("2023-w44", "Difference")

In [48]:
df_output_delta

Product_Cohort_Lables,Hero,High Value & Slow Moving,Low Value & Slow Moving,New Business Drivers,Repeat Business Drivers
User_Cohort_Lables,,,,,
Champions,2.000,-74665.000000,-115453.000,-79998.666667,-112649.25
Churned (Repeat Use),-144922.600,-127268.666667,-116348.000,-104754.250000,-105063.25
New,-97612.750,-129007.625000,-118857.875,-89507.125000,-88846.00
Recent & Loyals,-103612.875,-103761.875000,-95347.625,-87271.875000,-54864.25


In [49]:
df_output_Week.sum().sum()

1714868.0

In [50]:
df_output_delta.sum().sum()

-1949810.5583333333

In [51]:
delta_df = merged_dataframe[merged_dataframe["WeekCount"] == "2023-w44"]

In [52]:
selected_row = delta_df.loc[(delta_df['WeekCount'] == "2023-w44") & (delta_df['Variables'] == "Total_Value")]
selected_row

,WeekCount,Variables,Current Week Value,Last 8 Week Value,Tag,Difference
1554,2023-w44,Total_Value,1714868.0,1.949893e+06,Target_Variable,-235024.558333


In [53]:
df_output_Week

Product Cohort Lables,Hero,High Value & Slow Moving,Low Value & Slow Moving,New Business Drivers,Repeat Business Drivers
User_Cohort_Label,,,,,
Champions,74668.0,156240.0,9100.0,210621.0,273989.0
Churned (Repeat Use),39000.0,50832.0,23600.0,77148.0,96692.0
New,260243.0,129423.0,39183.0,33727.0,168903.0
Recent & Loyals,24784.0,9858.0,32062.0,998.0,3797.0
